In [1]:
import os
os.chdir("../")
%pwd

'/home/febin/MLOps-AWS-cancer-prediction-system'

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cancer_prediction.constants import *
from cancer_prediction.utils import read_yaml, create_directories


class ConfigurationManager:

    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):

        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        training_config = self.config.training
        data_ingestion_config = self.config.data_ingestion

        eval_config = ModelEvaluationConfig(
            path_of_model=Path(training_config.trained_model_path),
            training_data=os.path.join(data_ingestion_config.unzip_dir, 'data', 'chest-ct-scan-data'),
            mlflow_uri=None,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [11]:
import mlflow
import mlflow.keras
from pathlib import Path
import tensorflow as tf
from urllib.parse import urlparse
from cancer_prediction.utils import read_yaml, create_directories,save_json

In [12]:
class Evaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    configuration = ConfigurationManager()
    eval_config = configuration.get_model_evaluation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    #evaluation.log_into_mlflow()
    
except Exception as e:
    raise e

2025-04-08 18:57:50,084 - INFO __init__ - yaml file: config/config.yaml loaded successfully
2025-04-08 18:57:50,090 - INFO __init__ - yaml file: params.yaml loaded successfully
2025-04-08 18:57:50,092 - INFO __init__ - created directory at: artifacts
Found 78 images belonging to 2 classes.


2025-04-08 18:57:51.643957: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


5/5 [==============================] - 12s 2s/step - loss: 7.8927 - accuracy: 0.4359
2025-04-08 18:58:04,063 - INFO __init__ - json file saved at: scores.json
